In [ ]:
# --- Setup inicial ---
%reload_kedro

games = catalog.load("games")
teams = catalog.load("teams")


## 1. Selección de Datos

Nos enfocaremos en:
- `TEAM_ID` (equipo)
- `HOME_TEAM_ID`, `VISITOR_TEAM_ID`
- `PTS_home`, `PTS_away`
- Resultado del partido (ganó o perdió)

## 2. Limpieza de Datos
Eliminaremos filas con valores nulos en puntos.

In [ ]:
games_clean = games.dropna(subset=["PTS_home","PTS_away"])


## 3. Generación de Variables (Feature Engineering)

Creamos variables de interés:
- Diferencia de puntos (`POINT_DIFF`)
- Indicador de victoria (`HOME_WIN`, `AWAY_WIN`)


In [ ]:
games_clean["POINT_DIFF"] = games_clean["PTS_home"] - games_clean["PTS_away"]
games_clean["HOME_WIN"] = (games_clean["POINT_DIFF"] > 0).astype(int)
games_clean["AWAY_WIN"] = (games_clean["POINT_DIFF"] < 0).astype(int)


## 4. Métricas Agregadas por Equipo

Win rate de local y visitante:


In [ ]:
home_stats = games_clean.groupby("HOME_TEAM_ID").agg(
    games_home=("GAME_ID","count"),
    wins_home=("HOME_WIN","sum"),
    avg_points_home=("PTS_home","mean"),
    avg_points_against_home=("PTS_away","mean")
)
home_stats["win_rate_home"] = home_stats["wins_home"]/home_stats["games_home"]

away_stats = games_clean.groupby("VISITOR_TEAM_ID").agg(
    games_away=("GAME_ID","count"),
    wins_away=("AWAY_WIN","sum"),
    avg_points_away=("PTS_away","mean"),
    avg_points_against_away=("PTS_home","mean")
)
away_stats["win_rate_away"] = away_stats["wins_away"]/away_stats["games_away"]

display(home_stats.sort_values("win_rate_home",ascending=False).head())
display(away_stats.sort_values("win_rate_away").head())


> **Resultado obtenido:**  
> Un ranking de equipos que nos dice:
> - Mejores equipos en casa (win_rate_home más alto)
> - Peores equipos de visita (win_rate_away más bajo)
> - Equipos que anotan y reciben más puntos
